# строим витрины для аффинити 

In [26]:
import pandas as pd
# Папка с входными файлами и куда сохранять результаты
BASE_DIR = "/Users/anna_maltseva/Desktop/data"

stores = pd.read_csv(f"{BASE_DIR}/stores_zy_step1.csv")
rev_zy = pd.read_csv(f"{BASE_DIR}/reviews_zy_step2_normalized.csv")
auth_profiles = pd.read_csv(f"{BASE_DIR}/authors_zy_step3_profiles_v2.csv")
auth_reviews = pd.read_csv(f"{BASE_DIR}/authors_reviews_zy_step3_v2.csv")
auth_stats = pd.read_csv(f"{BASE_DIR}/authors_step3_author_stats.csv")

# приведение id к строкам, чтобы join не страдал
for df in [rev_zy, auth_profiles, auth_reviews, auth_stats]:
    df["author_id"] = df["author_id"].astype(str)

auth_reviews["place_org_id"] = auth_reviews["place_org_id"].astype("Int64")
rev_zy["org_id"] = rev_zy["org_id"].astype("Int64")


### Витрина 1: places_from_authors_step3 (все места, куда ходит аудитория бренда-фокуса)

In [27]:
zy_org_ids = set(rev_zy["org_id"].dropna().astype("Int64").unique())

auth_reviews["is_ZY"] = auth_reviews["place_org_id"].isin(zy_org_ids)

places = (
    auth_reviews
    .groupby(
        ["place_org_id", "place_name", "place_city", "place_category"],
        as_index=False
    )
    .agg(
        n_reviews=("review_datetime", "size"),
        n_authors=("author_id", "nunique"),
        avg_rating=("rating", "mean"),
        share_with_media=("media_total", lambda x: (x > 0).mean()),
        avg_media=("media_total", "mean"),
        avg_likes=("likes_count", "mean"),
        median_likes=("likes_count", "median"),
        avg_text_len=("text_raw", lambda s: s.fillna("").str.len().mean()),
        first_url=("place_url", "first"),
        is_ZY=("is_ZY", "max"),
    )
)

places.to_csv(f"{BASE_DIR}/places_from_authors_step3.csv", index=False)
places.head()


,place_org_id,place_name,place_city,place_category,n_reviews,n_authors,avg_rating,share_with_media,avg_media,avg_likes,median_likes,avg_text_len,first_url,is_ZY
0,266666840,Фазенда,Красноярск,Кафе,1,1,5.0,0.0,0.0,0.0,0.0,185.0,https://yandex.com/maps/org/fazenda/266666840/,False
1,1000046203,Стоматологический центр Алтос,Казань,Детская стоматология,1,1,5.0,0.0,0.0,0.0,0.0,1029.0,https://yandex.com/maps/org/stomatologicheskiy...,False
2,1000049023,Сбербанк России,Дзержинск,Банк,1,1,3.0,0.0,0.0,2.0,2.0,240.0,https://yandex.com/maps/org/sberbank/1000049023/,False
3,1000056889,T2,Нижний Новгород,Салон связи,1,1,5.0,0.0,0.0,0.0,0.0,127.0,https://yandex.com/maps/org/t2/1000056889/,False
4,1000063867,Башня 2000,Москва,Бизнес-центр,1,1,5.0,1.0,1.0,3.0,3.0,43.0,https://yandex.com/maps/org/bashnya_2000/10000...,False


Это витрина уровня место (точка): по каждому place_org_id — сколько авторов там были, как оценивают и т.д.

### Витрина 2: fact_author_place (факт-таблица автор–место)

In [28]:
fact_author_place = (
    auth_reviews
    .groupby(["author_id", "place_org_id"], as_index=False)
    .agg(
        n_reviews=("review_datetime", "size"),
        first_date=("review_datetime", "min"),
        last_date=("review_datetime", "max"),
        avg_rating=("rating", "mean"),
        any_media=("media_total", lambda x: int((x > 0).any())),
    )
)

fact_author_place["is_ZY"] = fact_author_place["place_org_id"].isin(zy_org_ids)

fact_author_place.to_csv(f"{BASE_DIR}/fact_author_place.csv", index=False)
fact_author_place.head()

,author_id,place_org_id,n_reviews,first_date,last_date,avg_rating,any_media,is_ZY
0,007ch87mf0ntraq18gj75dmg24,1144744287,1,2025-11-01,2025-11-01,1.0,0,False
1,007ch87mf0ntraq18gj75dmg24,9227686275,1,2025-04-01,2025-04-01,5.0,0,False
2,007ch87mf0ntraq18gj75dmg24,42690570156,1,2025-08-01,2025-08-01,5.0,0,False
3,007ch87mf0ntraq18gj75dmg24,112326370088,1,2025-08-01,2025-08-01,5.0,0,False
4,007ch87mf0ntraq18gj75dmg24,127131989637,1,2025-08-01,2025-08-01,5.0,0,False


Это самая важная таблица для аффинити: кто в какие бренды ходит.

### Витрина 3: brand_name (переход от точек к бренду)

Простой хелпер: бренд = «название до запятой»:

In [116]:
import numpy as np

def extract_brand(name):
    if pd.isna(name):
        return np.nan
    s = str(name)
    # можешь добавить свои правила сюда
    s = s.split("·")[0]
    s = s.split("•")[0]
    s = s.split(",")[0]
    return s.strip()

places["brand_name"] = places["place_name"].apply(extract_brand)

places.to_csv(f"{BASE_DIR}/places_from_authors_step3_with_brand.csv", index=False)
places[["place_name", "brand_name"]].head(30)

,place_name,brand_name
0,Фазенда,Фазенда
1,Стоматологический центр Алтос,Стоматологический центр Алтос
2,Сбербанк России,Сбербанк России
3,T2,T2
4,Башня 2000,Башня 2000
5,Santa Barbara Club,Santa Barbara Club
6,Драгоценная орхидея,Драгоценная орхидея
7,Artbanda,Artbanda
8,СберБанк,СберБанк
9,Мирная пристань,Мирная пристань


Агрегация до уровня бренда:

In [117]:
brands = (
    places
    .groupby("brand_name", as_index=False)
    .agg(
        n_places=("place_org_id", "nunique"),
        n_authors_total=("n_authors", "sum"),
        n_reviews_total=("n_reviews", "sum"),
        avg_rating=("avg_rating", "mean"),
        avg_share_with_media=("share_with_media", "mean"),
        is_ZY_brand=("is_ZY", "max"),
    )
)

brands.to_csv(f"{BASE_DIR}/brands_from_authors_step3.csv", index=False)
brands.head(20)


,brand_name,n_places,n_authors_total,n_reviews_total,avg_rating,avg_share_with_media,is_ZY_brand
0,#Prобки,1,1,1,5.000000,0.0,False
1,&Ruki,1,1,1,5.000000,0.0,False
2,+Медком,1,1,1,4.000000,0.0,False
3,0.75 Please,1,6,6,4.666667,0.5,False
4,007 CarWash,1,1,1,5.000000,0.0,False
5,1 Арт отель,1,2,2,4.500000,0.0,False
6,1 Копейка,1,1,1,2.000000,0.0,False
7,1 Оэр 2 МРЭО Госавтоинспекции ГУ МВД России по...,1,1,1,4.000000,0.0,False
8,1 Оэр МО ГИБДД ТНРЭР № 3 ГУ МВД России по г. М...,1,1,1,4.000000,0.0,False
9,1-2-3 Coffee Club,1,1,1,5.000000,0.0,False


Теперь у нас есть уровень бренда, а не только конкретных точек.

# Аффинити-анализ

In [155]:
import pandas as pd
import re

fact = pd.read_csv(
    f"{BASE_DIR}/fact_author_place.csv",
    dtype={"author_id": str, "place_org_id": str}
)

places = pd.read_csv(
    f"{BASE_DIR}/places_from_authors_step3_with_brand.csv",
    dtype={"place_org_id": str}
)

print(fact.columns)
print(places.columns)

Index(['author_id', 'place_org_id', 'n_reviews', 'first_date', 'last_date',
       'avg_rating', 'any_media', 'is_ZY'],
      dtype='object')
Index(['place_org_id', 'place_name', 'place_city', 'place_category',
       'n_reviews', 'n_authors', 'avg_rating', 'share_with_media', 'avg_media',
       'avg_likes', 'median_likes', 'avg_text_len', 'first_url', 'is_ZY',
       'brand_name'],
      dtype='object')


In [156]:
fact = fact.merge(
    places[["place_org_id", "brand_name", "is_ZY"]],
    on="place_org_id",
    how="left",
    suffixes=("", "_place")
)

In [157]:
if "is_ZY_place" in fact.columns:
    # если есть два флага, склеим их
    fact["is_ZY"] = fact["is_ZY"].fillna(fact["is_ZY_place"])
    fact.drop(columns=["is_ZY_place"], inplace=True)

fact["is_ZY"] = fact["is_ZY"].astype(bool)

In [158]:
BRAND_NORMALIZATION = {
    "Гипер Лента": "Лента",
    "Супер Лента": "Лента",
    "Магнит Семейный": "Магнит",
    "Магнит Экстра": "Магнит",
    "Чайхона № 1": "Vasilchuki Chaihona № 1",
    "Чайхона №1": "Vasilchuki Chaihona № 1",
}

fact["brand_name"] = fact["brand_name"].replace(BRAND_NORMALIZATION)

In [159]:
GENERIC_WORDS = {
    "пляж", "парк", "сквер", "набережная", "остановка",
    "магазин", "супермаркет", "гипермаркет", "столовая", "хинкальная",
    "центральный", "рынок", "аптека", "торговый", "центр",
    "смотровая", "площадка", "продукты", "автомойка", "шиномонтаж",
}

GENERIC_ANYWHERE = [
    "автостанция",
    "автовокзал",
    "железнодорожный вокзал",
    "жд вокзал",
    "вокзал",
    "автостоянка",
    "парковка",
    "аэропорт",
]

def mark_generic_brand(name: str) -> bool:
    if pd.isna(name):
        return False
    n = str(name).lower().strip()

    # 1) подстроки типа "аэропорт", "вокзал" и т.п.
    if any(kw in n for kw in GENERIC_ANYWHERE):
        return True

    # 2) токены из общих слов
    tokens = re.split(r"[ «»\"'.,\-]+", n)
    tokens = [t for t in tokens if t]

    if len(tokens) == 1 and tokens[0] in GENERIC_WORDS:
        return True

    if 1 <= len(tokens) <= 2 and all(tok in GENERIC_WORDS for tok in tokens):
        return True

    return False

fact["is_generic_place"] = fact["brand_name"].apply(mark_generic_brand)

In [160]:
KEEP_THESE = [
    "Главный вход ВДНХ",
    "Нижний парк Петергофа",
    "Ривьера-Сочи",
    "Воробьевы горы",
]

fact.loc[fact["brand_name"].isin(KEEP_THESE), "is_generic_place"] = False

In [163]:
fact[fact["brand_name"].str.contains("вокзал", case=False, na=False)][["brand_name","is_generic_place"]]

fact[fact["brand_name"].isin(KEEP_THESE)][["brand_name","is_generic_place"]]

,brand_name,is_generic_place
2621,Ривьера-Сочи,False
3383,Воробьевы горы,False
4530,Нижний парк Петергофа,False
4645,Главный вход ВДНХ,False
4890,Ривьера-Сочи,False
...,...,...
77186,Нижний парк Петергофа,False
77420,Воробьевы горы,False
78146,Ривьера-Сочи,False
78193,Главный вход ВДНХ,False


In [164]:
fact_clean = fact.loc[~fact["is_generic_place"].fillna(False)].copy()

In [165]:
N_ZY = fact_clean.loc[fact_clean["is_ZY"] == True, "author_id"].nunique()
print("N_ZY_total =", N_ZY)

N_ZY_total = 4490


In [166]:
brand_aff = (
    fact_clean
    .loc[fact_clean["is_ZY"] == False]   # внешние бренды
    .groupby("brand_name", as_index=False)
    .agg(
        n_authors        = ("author_id", "nunique"),
        n_author_reviews = ("n_reviews", "sum"),
        avg_rating_from_ZY = ("avg_rating", "mean"),
    )
)

brand_aff["N_ZY_total"] = N_ZY
brand_aff["share_of_ZY_audience"] = brand_aff["n_authors"] / N_ZY

print("Всего брендов в brand_aff:", len(brand_aff))
brand_aff.head()

Всего брендов в brand_aff: 36001


,brand_name,n_authors,n_author_reviews,avg_rating_from_ZY,N_ZY_total,share_of_ZY_audience
0,#Prобки,1,1,5.000000,4490,0.000223
1,&Ruki,1,1,5.000000,4490,0.000223
2,+Медком,1,1,4.000000,4490,0.000223
3,0.75 Please,6,6,4.666667,4490,0.001336
4,007 CarWash,1,1,5.000000,4490,0.000223


In [167]:
brand_aff.to_csv(
    f"{BASE_DIR}/brand_aff_full.csv",
    index=False,
    encoding="utf-8-sig"
)

In [168]:
MIN_AUTHORS = 20

top_brands_affinity_last = (
    brand_aff
    .loc[brand_aff["n_authors"] >= MIN_AUTHORS]
    .sort_values("share_of_ZY_audience", ascending=False)
    .reset_index(drop=True)
)

top_brands_affinity_last.to_csv(
    f"{BASE_DIR}/top_brands_affinity_last.csv",
    index=False,
    encoding="utf-8-sig"
)

In [174]:
# грузим аффинити и трафик
aff = pd.read_csv(f"{BASE_DIR}/top_brands_affinity_last.csv")
traffic = pd.read_csv(f"{BASE_DIR}/brands_traffic.csv")

# нормализация названий (то же, что мы делали в fact)
BRAND_NORMALIZATION = {
    "Гипер Лента": "Лента",
    "Супер Лента": "Лента",
    "Магнит Семейный": "Магнит",
    "Магнит Экстра": "Магнит",
    "Чайхона № 1": "Vasilchuki Chaihona № 1",
    "Чайхана № 1": "Vasilchuki Chaihona № 1",
    "Чайхона №1": "Vasilchuki Chaihona № 1",
}

aff["brand_name"] = aff["brand_name"].replace(BRAND_NORMALIZATION)
traffic["brand_name"] = traffic["brand_name"].replace(BRAND_NORMALIZATION)

# парсим ежемесячный трафик: "3.293M" в 3293000
def parse_visits(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace(" ", "")
    if s == "":
        return np.nan

    s = s.replace(",", ".")  # на всякий случай

    # ищем число + опциональный суффикс M/K
    m = re.match(r"^([0-9]*\.?[0-9]+)([MKmk]?)$", s)
    if not m:
        # если формат странный — просто пробуем как float
        try:
            return float(s)
        except ValueError:
            return np.nan

    val = float(m.group(1))
    suffix = m.group(2).upper()

    if suffix == "M":
        return val * 1_000_000
    elif suffix == "K":
        return val * 1_000
    else:
        # БЕЗ суффикса: трактуем как «сырое» число визитов
        return val

traffic["monthly_visits_num"] = traffic["monthly_visits"].apply(parse_visits)

# мерджим: только бренды из top_brands_affinity_last.
top_affinity_with_traffic = aff.merge(
    traffic[["brand_name", "domain", "monthly_visits_num"]],
    on="brand_name",
    how="left"   # все бренды из аффинити, трафик где есть, остальное NaN
)

# переименуем столбцы
top_affinity_with_traffic = top_affinity_with_traffic.rename(
    columns={
        "share_of_ZY_audience": "affinity",
        "monthly_visits_num": "monthly_visits",
    }
)

# считаем affinity_traffic = affinity * monthly_visits
top_affinity_with_traffic["affinity_traffic"] = (
    top_affinity_with_traffic["affinity"] *
    top_affinity_with_traffic["monthly_visits"]
)

# (опционально) нормированная версия, чтобы было красивее
top_affinity_with_traffic["affinity_traffic_norm"] = (
    top_affinity_with_traffic["affinity_traffic"] /
    top_affinity_with_traffic["affinity_traffic"].max()
)

# сохраняем
top_affinity_with_traffic.to_csv(
    f"{BASE_DIR}/top_affinity_with_traffic.csv",
    index=False,
    encoding="utf-8-sig"
)

## смотрим топ-20 брендов, отсортированных по аффинити-индексу 

In [176]:
top20_affinity = (
    top_affinity_with_traffic
    .sort_values("affinity", ascending=False)
    .head(20)
)
top20_affinity

,brand_name,n_authors,n_author_reviews,avg_rating_from_ZY,N_ZY_total,affinity,domain,monthly_visits,affinity_traffic,affinity_traffic_norm
0,Пятёрочка,518,756,4.052387,4490,0.115367,https://5ka.ru/,8.852000e+06,1.021233e+06,2.556499e-02
1,Wildberries,472,537,3.908411,4490,0.105122,https://www.wildberries.ru/,3.800000e+08,3.994655e+07,1.000000e+00
2,Магнит,389,524,4.036260,4490,0.086637,http://magnit.ru/,9.461000e+06,8.196724e+05,2.051923e-02
3,Ozon,317,355,4.225352,4490,0.070601,https://www.ozon.ru/,5.281000e+08,3.728457e+07,9.333614e-01
4,Вкусно — и точка,302,438,3.801370,4490,0.067261,https://vkusnoitochka.ru/,1.661000e+06,1.117198e+05,2.796733e-03
5,Перекрёсток,196,243,4.057613,4490,0.043653,https://www.perekrestok.ru/,4.107000e+06,1.792811e+05,4.488024e-03
6,СберБанк,192,232,4.220779,4490,0.042762,https://www.sberbank.ru/,5.058000e+07,2.162886e+06,5.414451e-02
7,Лента,174,211,4.426190,4490,0.038753,https://lenta.com/,6.789000e+06,2.630927e+05,6.586117e-03
8,Rostic's,167,216,3.430556,4490,0.037194,https://rostics.ru/,2.276000e+06,8.465301e+04,2.119157e-03
9,Красное&Белое,165,231,4.385281,4490,0.036748,https://krasnoeibeloe.ru/,2.166000e+06,7.959688e+04,1.992585e-03


## смотрим топ-20 брендов, отсортированных по аффинити-индексу с поправкой на траффик 

In [179]:
top20_affinity_traffic = (
    top_affinity_with_traffic
    .sort_values("affinity_traffic", ascending=False)
    .head(20)
)
top20_affinity_traffic

,brand_name,n_authors,n_author_reviews,avg_rating_from_ZY,N_ZY_total,affinity,domain,monthly_visits,affinity_traffic,affinity_traffic_norm
1,Wildberries,472,537,3.908411,4490,0.105122,https://www.wildberries.ru/,380000000.0,3.994655e+07,1.000000
3,Ozon,317,355,4.225352,4490,0.070601,https://www.ozon.ru/,528100000.0,3.728457e+07,0.933361
15,Яндекс Маркет,135,152,3.828947,4490,0.030067,https://market.yandex.ru/,145900000.0,4.386748e+06,0.109815
77,Авито,43,47,3.510638,4490,0.009577,https://www.avito.ru,324100000.0,3.103853e+06,0.077700
20,DNS,113,128,4.023438,4490,0.025167,https://www.dns-shop.ru/,89750000.0,2.258742e+06,0.056544
6,СберБанк,192,232,4.220779,4490,0.042762,https://www.sberbank.ru/,50580000.0,2.162886e+06,0.054145
0,Пятёрочка,518,756,4.052387,4490,0.115367,https://5ka.ru/,8852000.0,1.021233e+06,0.025565
2,Магнит,389,524,4.036260,4490,0.086637,http://magnit.ru/,9461000.0,8.196724e+05,0.020519
38,Альфа-Банк,67,80,3.675000,4490,0.014922,https://alfabank.ru,49800000.0,7.431180e+05,0.018603
12,CDEK,151,166,4.210843,4490,0.033630,https://www.cdek.ru/ru/,21880000.0,7.358307e+05,0.018420
